# 09 Giga-Scale Multimodal Biological Network

In this and in the following notebook, we provide a detailed look into how to construct very large multimodal networks. We do this by showing how to construct a giga-scale multimodal network of protein interactions for 2,031 species together with evolutionary information on orthologous genes.

Primary data sources for the network constructed here are the full [STRING](https://string-db.org/) and the full [GeneMANIA](http://genemania.org/) databases. 

**Note:** Because datasets in the two databases are several 10s of GBs in size, they cannot be included in this Github repository. 

**Note:** Users need to first download STRING data files from http://string-db.org/cgi/download.pl to a local machine, and place the files in a directory called `datasets/protein_example/string`. 

**Note:** Users need to download GeneMANIA data files from http://pages.genemania.org/data (more specifically, http://genemania.org/data/current) to a local machine, and place the files in a directory called `datasets/protein_example/genemania`. 

Assuming all data files are downloaded and available for analysis, we proceed with the network construction, representation, and analysis.

---

The figure below shows a conceptual structure of the multimodal network we are about to construct. 

The network has 2,032 modes, each corresponding to a distinct species and an additional mode for COGs (clusters of orthologous genes) and NOGs (non-supervised orthologous groups). The nodes within each mode represent proteins in the corresponding species. There are edges internal to each species mode representing different types of relationships between proteins (e.g., physical binding, co-ecpression, genetic interaction) in that species. Additionally, the network has edges that connect nodes in a species mode with nodes in the COG mode representing orthologous and paralogous relationships between proteins coming from different species.

<img src="figures/gigascale-multimodal.png" width="800">

To construct the network we follow the same five steps used in an earlier example on multimodal cancer network. That is, we use the exact same approach to construct, represent, and analyze the network as before, but the resulting network in this example is several orders of magnitude larger. 

The steps are:
- **Step 1**: Parse data
- **Step 2**: Create mode tables
- **Step 3**: Create link tables
- **Step 4**: Construct a multimodal network and save it on a disk
- **Step 5**: Load the network and perform analytics

# Step 1: Obtain and Parse Data

Because the raw data files is very large, the raw data must be obtained directly from the publically available databases as described in the introduction of this notebook.

In some cases, the raw data files need to be preprocessed and transformed into an appropriate format. This background step is detailed in notebook [10 Supplementary - Filtering a Giga-Scale Multimodal Biological Network](10 Supplementary - Filtering a Giga-Scale Multimodal Biological Network.ipynb).

# Step 2: Create Mode Tables

We construct STRING mode tables, one for each species in the STRING database and one additional mode for COGs and NOGs.

In [2]:
from collections import defaultdict
import os
import time

In [1]:
filename = "datasets/protein_example/string/STRING_v10/protein.aliases.v10.txt"
cog_filename = "datasets/protein_example/string/STRING_v10/COG.mappings.v10.txt"
output_dir = "output"

## 2.1 Create Species' Protein Mode Tables

We begin by reading STRING data file on protein name aliases and collecting all of protein names associated with each species. 

In [ ]:
seenTypes = defaultdict(set)
prevSpecies = 0
with open(filename, 'r') as f:
    for line in f:
        if line[0] == '#':
            continue
        splitLine = line.split('\t')
        currSpecies = splitLine[0].split('.')[0]
        seenTypes[currSpecies].add(splitLine[0])

For each species, we create a mode table for species-specific proteins (i.e., nodes) and save the table to a file.

In [ ]:
date = time.strftime("%Y%m%d")
db_id = 0

os.makedirs(os.path.join(output_dir, 'modes'))

for species in seenTypes:
    outfiletext = '# Full mode table for %s\n# File generated on: %s\n# mambo_nid\tdataset id\n' % (species, date)
    dbfiletext = '# Mode table for dataset: STRING\n# File generated on: %s\n# mambo_nid\tdataset_nid\n' % date
    outfilename = os.path.join(output_dir, 'modes',  'proteingene-%s-%s.tsv' % (species, date))
    dbfilename = os.path.join(output_dir, 'modes', 'proteingene-%s-0-STRING-%s.tsv' % (species, date))
    
    counter = 0
    for gene in seenTypes[species]:
        outfiletext += '%d\t%d\n' % (counter, db_id)
        dbfiletext +=  '%d\t%s\n' % (counter, gene)
        counter += 1
    
    with open(outfilename, 'w') as outF, open(dbfilename, 'w') as dbF:
        outF.write(outfiletext)
        dbF.write(dbfiletext)

## 2.2 Create COG Mode Tables

We read STRING data files on COG information and extract COG names from the files.

In [ ]:
seenTypes = set()
with open(cog_filename, 'r') as f:
    for line in f:
        if line[0] == '#':
            continue
        splitLine = line.split('\t')
        seenTypes.add(splitLine[3])

We create a mode table for COG names and save the table to a file.

In [ ]:
counter = 0
outfiletext = '# Full mode table for COGs\n# File generated on: %s\n# mambo_nid\tdataset id\n' % date
dbfiletext = '# Mode table for dataset: STRING\n# File generated on: %s\n# mambo_nid\tdataset_nid\n' % date
outfilename = os.path.join(output_dir, 'modes', 'proteingene-COG-%s.tsv' % date)
dbfilename = os.path.join(output_dir, 'modes', 'proteingene-COG-0-STRING-%s.tsv' % date)

for cog in seenTypes:
    outfiletext += '%d\t%d\n' % (counter, db_id)
    dbfiletext +=  '%d\t%s\n' % (counter, cog)
    counter += 1
    
with open(outfilename, 'w') as outF, open(dbfilename, 'w') as dbF:
    outF.write(outfiletext)
    dbF.write(dbfiletext)

# Step 3: Create Link Tables for the Giga-Scale Multimodal network

There are three main groups of link types in the giga-scale multimodal network. 

(1) First, there are COG link types, which go between protein nodes in a given species and the COG nodes.

(2) Second, there are link types that specify relationship between proteins in each species. These link types encode protein-protein relationships in a form of 8 distinct relationship variants (as defined by [STRING database](https://string-db.org/)): neighborhood, fusion, cooccurence, homology, coexpression, experiments, database, text mining. 

(3) Third, there are link types that come from [GeneMANIA database](http://genemania.org/), of which there are six variants. One variant, co-expression, is shared with the STRING database, but other 5 variants are unique to data in GeneMANIA: co-localization, genetic interactions, pathway membership, physical interactions, and predicted interactions.

We first import relevant Python packages.

In [4]:
import os

from utils.create_mambo_crossnet_table import create_mambo_crossnet_table

We specify directories where data files are located.

In [ ]:
cog_data_dir = "datasets/protein_example/string/COG"
string_data_dir = "datasets/protein_example/string"
genemania_data_dir = "datasets/protein_example/genemania"
mode_data_dir = "output/modes"
output_dir = "output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

cog_output_dir = "output/cog_links"
if not os.path.exists(cog_output_dir):
    os.makedirs(cog_output_dir)

## 3.1 Create Protein-COG Link Tables

For each species, we create a link table between the mode representing the species and the mode representing orthologous groups. 

These link tables correspond to thick black lines in the figure below.

<img src="figures/gigascale-multimodal.png" width="800">

In [ ]:
dst_file = os.path.join(mode_data_dir, "proteingene-COG-0-STRING-20170615.tsv")
for filename in os.listdir(cog_data_dir):
    filepath = os.path.join(cog_data_dir, filename)
    datasetname = filename.split('-')[2].split('.')[0]
    srcfile = os.path.join(mode_data_dir, "proteingene-%s-0-STRING-20170615.tsv" % datasetname)
                           
    create_mambo_crossnet_table(input_file=filepath, 
                               src_file=srcfile, 
                               dst_file=dst_file, 
                               dataset_name=datasetname,
                               db_id=0, 
                               src_node_index=0, 
                               dst_node_index=1, 
                               mode_name1=None,
                               mode_name2=None, 
                               output_dir=cog_output_dir, 
                               full_crossnet_file=None, 
                               db_edge_file=None,
                               src_mode_filter=None, 
                               dst_mode_filter=None, 
                               mambo_id_counter_start=0,
                               skip_missing_ids=False)

## 3.2 Create Protein-Protein Link Tables based on STRING Database

We create link tables representing protein-protein interactions available in the STRING database. 

These link tables correspond to thick curved lines in the figure below.

<img src="figures/gigascale-multimodal.png" width="800">

In [ ]:
types = [
'neighborhood',
'fusion',
'cooccurence',
'coexpression',
'experiments',
'database',
'textmining',
'combined_score',
]

for t in types:
    type_dir = os.path.join(string_data_dir, t)
    type_output_dir = os.path.join(output_dir, t + '_links')
    os.makedirs(type_output_dir)
    
    for filename in os.listdir(type_dir):
        filepath = os.path.join(type_dir, filename)
        datasetname = filename.split('-')[1].split('.')[0]
        srcfile = os.path.join(mode_data_dir, "proteingene-%s-0-STRING-20170615.tsv" % datasetname)
        
        create_mambo_crossnet_table(input_file=filepath, 
                           src_file=srcfile, 
                           dst_file=srcfile,
                           dataset_name=datasetname,
                           db_id=0, 
                           src_node_index=0, 
                           dst_node_index=1, 
                           mode_name1="proteingene",
                           mode_name2="proteingene", 
                           output_dir=type_output_dir, 
                           full_crossnet_file=None, 
                           db_edge_file=None,
                           src_mode_filter=None, 
                           dst_mode_filter=None, 
                           mambo_id_counter_start=0,
                           skip_missing_ids=False)

## 3.3 Create Protein-Protein Link Tables based on GeneMANIA Database

We create link tables representing protein-protein interactions available in the GeneMANIA database. 

These link tables correspond to thick curved lines in the figure below. 

*Note:* GeneMANIA does not have any information on links *between* species. Links between species are exclusively provided by the Protein-COG links that we created first.

<img src="figures/gigascale-multimodal-2.png" width="400">

In [ ]:
types = ["Co-expression", "Co-localization", "Genetic_interactions", 
         "Pathway", "Physical_interactions", "Predicted"]

dirname = {"Co-localization" : "colocalization_crossnet", 
           "Co-expression" : "coexpression_crossnet",
           "Genetic_interactions" : "genetic_interactions_crossnet",
           "Pathway": "pathway_crossnet",
           "Physical_interactions" : "physical_interactions_crossnet",
           "Predicted" : "predicted_crossnet"}

species = ["Arabidopsis_thaliana", "Caenorhabditis_elegans", "Danio_rerio", 
           "Drosophila_melanogaster", "Escherichia_coli", "Homo_sapiens",
           "Mus_musculus", "Rattus_norvegicus", "Saccharomyces_cerevisiae"]

for d in dirname.values():
    if not os.path.exists(d):
        os.makedirs(d)

for s in species:
    species_directory = os.path.join(genemania_data_dir, s)
    typecount = {"Co-localization" : 0, 
                 "Co-expression" : 0,
                 "Genetic_interactions" : 0,
                 "Pathway" : 0,
                 "Physical_interactions" : 0,
                 "Predicted" : 0}
    
    for filename in os.listdir(species_directory):
        if any(typename in filename for typename in types):
            filename = filename.replace("'", "\\'")
            filepath = os.path.join(directory, filename)
            datasetname = filename.split('.')[1]
            filetype = filename.split('.')[0]
            srcfile = os.path.join(mode_data_dir, "proteingene-%s-1-GENEMANIA-20170523.tsv" % datasetname)
            type_output_dir = os.makedirs(os.path.join(output_dir, dirname[filetype]))
            
            create_mambo_crossnet_table(input_file=filepath, 
                    src_file=srcfile, 
                   dst_file=srcfile,
                   dataset_name=datasetname,
                   db_id=typecount[filetype], 
                   src_node_index=0, 
                   dst_node_index=1, 
                   mode_name1=None,
                   mode_name2=None, 
                   output_dir=type_output_dir, 
                   full_crossnet_file=None, 
                   db_edge_file=None,
                   src_mode_filter=None, 
                   dst_mode_filter=None, 
                   mambo_id_counter_start=0,
                   skip_missing_ids=False)
            typecount[filetype] += 1

# Step 4: Construct the Giga-Scale Multimodal Network

Next, we show how to put together all mode and link tables we have just created in order to construct a network.

We start by importing the required packages.

In [5]:
import os

import snap
from utils.network_utils import load_mode_to_graph, load_crossnet_to_graph

In [ ]:
mode_data_dir = "output/modes"
cog_cross_dir = "output/cog_links"
cross_base_dir = "output"

output_dir = "output"
graph_name = "protein_example.graph"

## 4.1 Build the Multimodal Network

Start by creating an empty multimodal network.

In [ ]:
context = snap.TTableContext()
Graph = snap.TMMNet.New()

#### Load modes

Add all modes to the network. 

In [ ]:
for f in os.listdir(mode_data_dir):
    splitName = f.split('-')
    if len(splitName) == 3:
        filepath = os.path.join(mode_data_dir, f)
        load_mode_to_graph(splitName[1], filepath, Graph, context)

#### Load Protein-COG link tables

Add all link types to the network. 

In [ ]:
for f in os.listdir(cog_cross_dir):
    splitName = f.split('-')
    if len(splitName) == 4:
        edgeId = "Cog%sId" % splitName[1]
        srcName = splitName[1]
        dstName = "COG"
        filepath = os.path.join(cog_cross_dir, f)
        load_crossnet_to_graph(context, edgeId, srcName, dstName, filepath, Graph, prefix="COG")

#### Load the remaining link types representing protein-protein interactions within each species mode

In [ ]:
types = [
'neighborhood',
'fusion',
'cooccurence',
'homology',
'coexpression', 
'experiments',
'database',
'textmining',
]

for t in types:
    directory = os.path.join(cross_base_dir, t + '_links')
    for f in os.listdir(directory):
        splitName = f.split('-')
        if len(splitName) == 4:
            edgeId = splitName[1] + '-' + splitName[1] + 'Id'
            srcName = splitName[1]
            dstName = splitName[1]
            filepath = os.path.join(directory, f)
            load_crossnet_to_graph(context, edgeId, srcName, dstName, filepath, Graph, prefix=t)

#### Save the multimodal network to a disk for later use

In [ ]:
outputPath = os.path.join(output_dir, graph_name)
FOut = snap.TFOut(outputPath)
Graph.Save(FOut)
FOut.Flush()

# Step 5: Load the Giga-Scale Multimodal Network and Perform Analytics

Let us import relevant packages.

In [8]:
import snap

from utils.network_utils import get_num_elem_per_mode, get_num_elem_per_link

## 5.1 Load the Giga-Scale Multimodal Network

We describe how the multimodal network can be loaded from a disk.

In [7]:
filename = "output/protein_example.graph"
FIn = snap.TFIn(filename)
Graph = snap.TMMNet.Load(FIn)

To make sure the network has been loaded correctly, let us determine the number of modes and the number of entities for each mode.

In [ ]:
print 'Modes: %d' % Graph.GetModeNets()

mode_num_elem = get_num_elem_per_mode(Graph)

# there are many modes, print information for `num_to_display` number of modes
num_to_display = 10
items = [(k, mode_num_elem[k]) for k in sorted(mode_num_elem.keys())[:10]]
print items

Let us also determine the number of link types and the number of edges for each link type.

In [ ]:
print 'Link types: %d' % Graph.GetCrossNets()

link_num_elem = get_num_elem_per_link(Graph)

# there are many link types, print information for every `display_every` link type
display_every = 200
items = [(k, link_num_elem[k]) for k in sorted(link_num_elem.keys())[::display_every]]
print '\n'.join('{} = {}'.format(*link) for link in items)

## 5.2 Perform Analytics

We convert the network to a directed graph and perform analytics. 

*Note:* The network can be analyzed as a directed graph ([TNGraph](https://snap.stanford.edu/snappy/doc/reference/graphs.html#TNGraph)), undirected graph ([TUNGraph](https://snap.stanford.edu/snappy/doc/reference/graphs.html#tungraph)), or attributed graph ([PNEANet](https://snap.stanford.edu/snappy/doc/tutorial/tutorial.html#snap-types-in-snap-py)).

In [9]:
import snap
import time

from utils.network_utils import get_num_elem_per_mode

We load the network from a file.

In [ ]:
filename = "output/protein_example.graph"
FIn = snap.TFIn(filename)
Graph = snap.TMMNet.Load(FIn)

#### Print the number of modes and links to check the network is loaded correctly

In [ ]:
print 'Modes: %d' % Graph.GetModeNets()
print 'Link types: %d' % Graph.GetCrossNets()

#### Convert the network to a directed network

In [ ]:
crossnetids = snap.TIntV()
crossneti = Graph.BegCrossNetI()
while crossneti < Graph.EndCrossNetI():
    crossnetids.Add(crossneti.GetCrossId())
        
nodeattrmapping = snap.TIntStrStrTrV()
edgeattrmapping = snap.TIntStrStrTrV()
    
start_time = time.time()
DirectedNetwork = Graph.ToNetwork(crossnetids, nodeattrmapping, edgeattrmapping)
end_time = time.time()
print "Conversion to TNEANet takes %s seconds" % (end_time - start_time)

#### Compute and display network statistics

In [ ]:
snap.PrintInfo(DirectedNetwork, "Python type PNEANet", "output/output.txt", True)

In [ ]:
map(lambda x: x.replace("\n", ""), open("output/output.txt").readlines())

#### Calculate network diameter

In [ ]:
print "Diameter: %d" % snap.GetBfsFullDiam(DirectedNetwork, 10)

#### Calculate size distribution of weakly connected components

In [ ]:
CntV = snap.TIntPrV()
snap.GetWccSzCnt(DirectedNetwork, CntV)
sizestring = ""
for p in CntV:
    sizestring += "%d\t%d\n" % (p.GetVal1(), p.GetVal2())
print sizestring